In [2]:
import pandas as pd

In [9]:
train_data = pd.read_csv('data\ViFactCheck\\train_data.csv')
dev_data = pd.read_csv('data\ViFactCheck\dev_data.csv')
test_data = pd.read_csv('data\ViFactCheck\\test_data.csv')

In [11]:
train_data['labels'].value_counts()

labels
0    1751
1    1658
2    1653
Name: count, dtype: int64

In [12]:
dev_data['labels'].value_counts()

labels
0    256
1    244
2    223
Name: count, dtype: int64

In [10]:
test_data['labels'].value_counts()

labels
0    508
2    471
1    468
Name: count, dtype: int64

In [14]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5062 entries, 0 to 5061
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         5062 non-null   int64 
 1   index              5062 non-null   int64 
 2   Statement          5062 non-null   object
 3   Context            5062 non-null   object
 4   annotation_id      5062 non-null   int64 
 5   Topic              5062 non-null   object
 6   Author             5062 non-null   object
 7   Url                5062 non-null   object
 8   labels             5062 non-null   int64 
 9   Evidence           5062 non-null   object
 10  splited_sentences  5062 non-null   object
 11  Evidence_List      5062 non-null   object
 12  evidence_top5      5062 non-null   object
 13  len_evidence       5062 non-null   int64 
dtypes: int64(5), object(9)
memory usage: 553.8+ KB


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

# Preprocess the data
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length")

tokenizer = AutoTokenizer.from_pretrained("bert-base-vietnamese-cased")
tokenized_dataset = data.map(preprocess_function, batched=True)

# Split the dataset into training and validation sets
train_dataset, eval_dataset = tokenized_dataset.train_test_split(test_size=0.2)

# Load the pre-trained model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-vietnamese-cased")

# Fine-tune the model
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()

In [13]:
pip install underthesea

   ---------------------------------------- 0.0/20.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/20.9 MB 1.3 MB/s eta 0:00:17
   ---------------------------------------- 0.1/20.9 MB 1.1 MB/s eta 0:00:19
   ---------------------------------------- 0.2/20.9 MB 1.6 MB/s eta 0:00:14
    --------------------------------------- 0.3/20.9 MB 2.3 MB/s eta 0:00:10
    --------------------------------------- 0.3/20.9 MB 2.3 MB/s eta 0:00:10
    --------------------------------------- 0.4/20.9 MB 1.6 MB/s eta 0:00:13
    --------------------------------------- 0.4/20.9 MB 1.4 MB/s eta 0:00:15
    --------------------------------------- 0.4/20.9 MB 1.4 MB/s eta 0:00:15
    --------------------------------------- 0.4/20.9 MB 1.4 MB/s eta 0:00:15
    --------------------------------------- 0.4/20.9 MB 1.4 MB/s eta 0:00:15
    --------------------------------------- 0.4/20.9 MB 1.4 MB/s eta 0:00:15
   - -------------------------------------- 0.8/20.9 MB 1.4 MB/s eta 0:00:15
   - -

In [ ]:
def processing_evidence(data):
    data['evidence_train'] = ""  # Tạo cột 'evidence_train' trước
    for i in tqdm(range(len(data))):
        if data['verdict'][i] == "NEI":
            data.loc[i, 'evidence_train'] = data['context'][i].split('. ')[0]
        else:
            data.loc[i, 'evidence_train'] = data['evidence'][i]
    return data

In [ ]:
import underthesea
from scipy.spatial.distance import cosine
import sys
import os
import re
import json
import numpy as np

def preprocess_text(text: str) -> str:
    text = re.sub(r"['\",\.\?:\-!]", "", text)
    text = text.strip()
    text = " ".join(text.split())
    text = text.lower()
    return text

def preprocess_text2(text: str) -> str:
    text = re.sub(r"['\"\.]", "", text)
    text = text.strip()
    text = " ".join(text.split())
    text = text.lower()
    return text

def move_punctuation_after_quotes(text):
    pattern = r'([.?!])(”)'
    result = re.sub(pattern, r'\2\1', text)
    return result

def split_context(text):
    # text = move_punctuation_after_quotes(text)
    punctuation_marks = r'(?

def preprocess(text):
  text = preprocess_text2(text)
  text = move_punctuation_after_quotes(text)

  text = str(text)\
    .replace(";", "").replace("“", "") \
    .replace(":", "").replace("”", "") \
    .replace('"', "").replace("'", "") \
    .replace("!", "").replace("?", "") \
    .replace("-", "")
  text = text.strip().lower()
  return text